In [ ]:
from ariautils.midi import MidiDict
import torch
from transformers import AutoTokenizer
from src.model.model import MidiAria

tokenizer = AutoTokenizer.from_pretrained(
    "loubb/aria-medium-base",
    trust_remote_code=True,
    add_eos_token=True,
    add_dim_token=False
)
checkpoint_path = "../checkpoints/aria/aria-harmony-epoch=03-val_loss=4.4451.ckpt"
model = MidiAria(tokenizer, None)
model.to_lora()
state_dict = torch.load(checkpoint_path, map_location=torch.device('cpu'))['state_dict']
model.load_state_dict(state_dict)


device = "mps" if torch.backends.mps.is_available() else "cpu"
torch.Tensor.cuda = lambda self, *args, **kwargs: self.to(device)


PROMPT_MIDI_LOAD_PATH = "../data/test/test_melody.mid"
CONTINUATION_MIDI_SAVE_PATH = "../data/test/continuation.midi"
midi_dict = MidiDict.from_midi(PROMPT_MIDI_LOAD_PATH)
tokens = tokenizer.tokenize(midi_dict, add_eos_token=True, add_dim_token=True)
token_ids = tokenizer._tokenizer.encode(tokens)
prompt_input_ids = torch.tensor([token_ids], device='cpu')

continuation = model.model.generate(
    prompt_input_ids.to('cpu'),
    max_length=2048,
    do_sample=True,
    temperature=0.97,
    top_p=0.95,
    use_cache=True,
)


prompt_len = prompt_input_ids.shape[1]
generated_tokens = continuation[0][prompt_len:]  # only the new part

# decode just the continuation
midi_dict = tokenizer.decode(generated_tokens.tolist())
print(tokenizer._tokenizer.decode(generated_tokens.tolist()))
midi_dict.to_midi().save(CONTINUATION_MIDI_SAVE_PATH)

print(f"Saved continuation to {CONTINUATION_MIDI_SAVE_PATH}")
